Lambda School Data Science

*Unit 2, Sprint 3, Module 3*

---


# Permutation & Boosting

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] If you haven't completed assignment #1, please do so first.
- [ ] Continue to clean and explore your data. Make exploratory visualizations.
- [ ] Fit a model. Does it beat your baseline? 
- [ ] Try xgboost.
- [ ] Get your model's permutation importances.

You should try to complete an initial model today, because the rest of the week, we're making model interpretation visualizations.

But, if you aren't ready to try xgboost and permutation importances with your dataset today, that's okay. You can practice with another dataset instead. You may choose any dataset you've worked with previously.

The data subdirectory includes the Titanic dataset for classification and the NYC apartments dataset for regression. You may want to choose one of these datasets, because example solutions will be available for each.


## Reading

Top recommendations in _**bold italic:**_

#### Permutation Importances
- _**[Kaggle / Dan Becker: Machine Learning Explainability](https://www.kaggle.com/dansbecker/permutation-importance)**_
- [Christoph Molnar: Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/feature-importance.html)

#### (Default) Feature Importances
  - [Ando Saabas: Selecting good features, Part 3, Random Forests](https://blog.datadive.net/selecting-good-features-part-iii-random-forests/)
  - [Terence Parr, et al: Beware Default Random Forest Importances](https://explained.ai/rf-importance/index.html)

#### Gradient Boosting
  - [A Gentle Introduction to the Gradient Boosting Algorithm for Machine Learning](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/)
  - [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/ISLR%20Seventh%20Printing.pdf), Chapter 8
  - _**[Gradient Boosting Explained](https://www.gormanalysis.com/blog/gradient-boosting-explained/)**_ — Ben Gorman
  - [Gradient Boosting Explained](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html) — Alex Rogozhnikov
  - [How to explain gradient boosting](https://explained.ai/gradient-boosting/) — Terence Parr & Jeremy Howard

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import category_encoders as ce
df = sns.load_dataset('penguins')
df.head()
droprows = df.T.isnull().sum().sort_values(ascending=False).head(2).index

In [2]:
target = 'species'
features = df.columns.drop(target).tolist()
df.shape
train, test = train_test_split(df, train_size=0.80, test_size=0.20, random_state=8)
df.isnull().sum()

species               0
island                0
culmen_length_mm      2
culmen_depth_mm       2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

In [7]:
df[target].value_counts(normalize=True)
#majority class baseline = 44%

Adelie       0.441860
Gentoo       0.360465
Chinstrap    0.197674
Name: species, dtype: float64

In [8]:
def wrangle(X):
  X= X.copy()
  X.sex = X.sex.fillna(value='UNK')
  droprows = X.T.isnull().sum().sort_values(ascending=False).head(2).index   ## two rows that have all 5 nans 
  X.drop(index=droprows, inplace=True)
  return X

X_train = wrangle(train)[features]
y_train = wrangle(train)[target]

X_test = wrangle(test)[features]
y_test = wrangle(test)[target]

X_train.shape,y_train.shape,X_test.shape
                       

((273, 6), (273,), (67, 6))

In [11]:
from sklearn.tree import DecisionTreeClassifier

from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, recall_score


he = ce.OneHotEncoder()
X_enc = he.fit_transform(X_train)
X_test_enc  = he.transform(X_test)
dtc_model = DecisionTreeClassifier(max_depth=2)
dtc_model.fit(X_enc, y_train)
y_pred = dtc_model.predict(X_test_enc)
print('test accuracy', accuracy_score(y_test, y_pred))

test accuracy 0.9850746268656716


In [12]:
import eli5
from eli5.sklearn  import PermutationImportance

perm = PermutationImportance(dtc_model, random_state=887).fit(X_test_enc,y_test)
eli5.show_weights(perm, feature_names = X_test_enc.columns.tolist())


/home/dliu/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dliu/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


Weight,Feature
0.4090 ± 0.0670,flipper_length_mm
0.2627 ± 0.1343,culmen_length_mm
0.1761 ± 0.0637,culmen_depth_mm
0 ± 0.0000,sex_3
0 ± 0.0000,sex_2
0 ± 0.0000,sex_1
0 ± 0.0000,body_mass_g
0 ± 0.0000,island_3
0 ± 0.0000,island_2
0 ± 0.0000,island_1


In [15]:
from xgboost import XGBClassifier
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    XGBClassifier(n_estimators=100, random_state=42, n_jobs=-1)
)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print('Test Accuracy', accuracy_score(y_test, y_pred))


Test Accuracy 0.9850746268656716
